In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import os

/Users/koretsunobuyasu/.pyenv/versions/3.7.5/lib/python3.7/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
DIR = f'../data/original/{datetime.now().strftime("%Y-%m-%d")}'
CASES = '/cases_total.csv'
DEATH = '/death_total.csv'
PCR_POSITIVE = '/pcr_positive_daily.csv'
PCR_TESTED = '/pcr_tested_daily.csv'
CURRENT = '/current_situation.csv'

In [3]:
%%time
print('start')
cases_df = pd.read_csv(DIR + CASES)
death_df = pd.read_csv(DIR + DEATH)
positive_df = pd.read_csv(DIR + PCR_POSITIVE)
tested_df = pd.read_csv(DIR + PCR_TESTED)
current_df = pd.read_csv(DIR + CURRENT)

start
CPU times: user 12.7 ms, sys: 3.7 ms, total: 16.4 ms
Wall time: 15.6 ms


In [4]:
%%time
print('start')
display(cases_df.head())
display(death_df.head())
display(positive_df.head())
display(tested_df.head())
display(current_df.head())

start


,日付,入院治療を要する者
0,2020/2/4,15
1,2020/2/5,16
2,2020/2/6,12
3,2020/2/7,12
4,2020/2/8,7


,日付,死亡者数
0,2020/2/14,1
1,2020/2/15,1
2,2020/2/16,1
3,2020/2/17,1
4,2020/2/18,1


,日付,PCR 検査陽性者数(単日)
0,2020/1/16,1
1,2020/1/17,0
2,2020/1/18,0
3,2020/1/19,0
4,2020/1/20,0


,日付,PCR 検査実施件数(単日)
0,2020/2/5,4
1,2020/2/6,19
2,2020/2/7,9
3,2020/2/8,4
4,2020/2/9,10


,Unnamed: 0,PCR検査 実施人数 ※3,陽性者数,入院治療等を要する者の数,うち重症者の数,退院又は療養解除となった者の数,死亡者数,確認中
0,"国内事例 ※1,※5(チャーター便帰国者を除く)","648,773(+18,086)","31,333(+968) ※2","7,063(+235)",81(+5),"23,256(+696)","1,000(+3)",22(-5)
1,空港検疫,"115,744(+1,031)",553(+13),316(+13),0,236,1,0
2,チャーター便帰国者事例,829,15,0,0,15,0,0
3,合計,"765,346(+19,117)","31,901(+981) ※2","7,379(+248)",81(+5),"23,507(+696)","1,001(+3)",22(-5)


CPU times: user 40.1 ms, sys: 4.74 ms, total: 44.8 ms
Wall time: 47.5 ms


In [5]:
%%time
current_df = current_df.rename(columns={'Unnamed: 0': '感染経路','PCR検査\n実施人数 ※3':'PCR検査 実施人数','退院又は療養解除と\nなった者の数': '退院または療養解除となった者の数','確認中 ※4': '確認中'})
current_df


CPU times: user 988 µs, sys: 61 µs, total: 1.05 ms
Wall time: 1.03 ms


,感染経路,PCR検査 実施人数 ※3,陽性者数,入院治療等を要する者の数,うち重症者の数,退院又は療養解除となった者の数,死亡者数,確認中
0,"国内事例 ※1,※5(チャーター便帰国者を除く)","648,773(+18,086)","31,333(+968) ※2","7,063(+235)",81(+5),"23,256(+696)","1,000(+3)",22(-5)
1,空港検疫,"115,744(+1,031)",553(+13),316(+13),0,236,1,0
2,チャーター便帰国者事例,829,15,0,0,15,0,0
3,合計,"765,346(+19,117)","31,901(+981) ※2","7,379(+248)",81(+5),"23,507(+696)","1,001(+3)",22(-5)


In [6]:
%%time
print('start')
# caseとdeathを結合
print(len(cases_df))
print(len(death_df))
final_df = pd.merge(cases_df, death_df, on='日付', how='left')
final_df = pd.merge(final_df, positive_df, on='日付', how='left')
final_df = pd.merge(final_df, tested_df, on='日付', how='left')
display(final_df)

start
176
166


,日付,入院治療を要する者,死亡者数,PCR 検査陽性者数(単日),PCR 検査実施件数(単日)
0,2020/2/4,15,NaN,2,NaN
1,2020/2/5,16,NaN,2,4.0
2,2020/2/6,12,NaN,0,19.0
3,2020/2/7,12,NaN,0,9.0
4,2020/2/8,7,NaN,0,4.0
...,...,...,...,...,...
171,2020/7/24,5949,992.0,766,5695.0
172,2020/7/25,6315,995.0,798,9530.0
173,2020/7/26,6573,995.0,830,3785.0
174,2020/7/27,6828,997.0,581,24182.0


CPU times: user 30.5 ms, sys: 3.97 ms, total: 34.5 ms
Wall time: 33.3 ms


In [7]:
%%time
current_df.replace('\n', ',')

CPU times: user 692 µs, sys: 57 µs, total: 749 µs
Wall time: 984 µs


,感染経路,PCR検査 実施人数 ※3,陽性者数,入院治療等を要する者の数,うち重症者の数,退院又は療養解除となった者の数,死亡者数,確認中
0,"国内事例 ※1,※5(チャーター便帰国者を除く)","648,773(+18,086)","31,333(+968) ※2","7,063(+235)",81(+5),"23,256(+696)","1,000(+3)",22(-5)
1,空港検疫,"115,744(+1,031)",553(+13),316(+13),0,236,1,0
2,チャーター便帰国者事例,829,15,0,0,15,0,0
3,合計,"765,346(+19,117)","31,901(+981) ※2","7,379(+248)",81(+5),"23,507(+696)","1,001(+3)",22(-5)


In [8]:
%%time
PREFECTURE = '/prefectures.csv'
prefecture_df = pd.read_csv(DIR + PREFECTURE)

CPU times: user 12 ms, sys: 3.73 ms, total: 15.8 ms
Wall time: 16.5 ms


In [9]:
%%time
display(prefecture_df)

,year,month,date,prefectureNameJ,prefectureNameE,testedPositive,peopleTested,discharged,deaths
0,2020,3,11,北海道,Hokkaido,118,1069.0,NaN,NaN
1,2020,3,11,青森県,Aomori,0,58.0,NaN,NaN
2,2020,3,11,岩手県,Iwate,0,20.0,NaN,NaN
3,2020,3,11,宮城県,Miyagi,1,94.0,NaN,NaN
4,2020,3,11,秋田県,Akita,2,40.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
6575,2020,7,28,熊本県,Kumamoto,114,4607.0,47.0,3.0
6576,2020,7,28,大分県,Oita,62,6312.0,59.0,1.0
6577,2020,7,28,宮崎県,Miyazaki,85,2970.0,20.0,0.0
6578,2020,7,28,鹿児島県,Kagoshima,219,9768.0,138.0,0.0


CPU times: user 12.8 ms, sys: 1.51 ms, total: 14.3 ms
Wall time: 14.1 ms


In [10]:
%%time
final_df = final_df.fillna(0)
final_df.rename(columns={'日付':'date','入院治療を要する者': 'hospitalization_required', '死亡者数': 'number_of_death', 'PCR 検査陽性者数(単日)': 'pcr_number_of_positives__single_day','PCR 検査実施件数(単日)': 'pcr_number_of_cases__single_day'}, inplace=True)
final_df['number_of_death'] = final_df['number_of_death'].astype(int)
final_df['pcr_number_of_cases__single_day'] = final_df['pcr_number_of_cases__single_day'].astype(int)
final_df

CPU times: user 3.13 ms, sys: 432 µs, total: 3.57 ms
Wall time: 3.67 ms


,date,hospitalization_required,number_of_death,pcr_number_of_positives__single_day,pcr_number_of_cases__single_day
0,2020/2/4,15,0,2,0
1,2020/2/5,16,0,2,4
2,2020/2/6,12,0,0,19
3,2020/2/7,12,0,0,9
4,2020/2/8,7,0,0,4
...,...,...,...,...,...
171,2020/7/24,5949,992,766,5695
172,2020/7/25,6315,995,798,9530
173,2020/7/26,6573,995,830,3785
174,2020/7/27,6828,997,581,24182


In [11]:
%%time
from datetime import datetime
print('start')
datetime.now().strftime('%Y-%m-%d')

start
CPU times: user 178 µs, sys: 56 µs, total: 234 µs
Wall time: 300 µs


'2020-07-29'

In [12]:
%%time
os.chdir('../datasource/')
import base_data as bd
os.chdir('../notebooks')
bd.basedata_to_pickle()

CPU times: user 49.5 ms, sys: 5.97 ms, total: 55.4 ms
Wall time: 57.2 ms
